In [1]:
import os

# 1. Extract Keypoints
load video data and extract keypoints using mediapose into /data/pose_data.csv

In [2]:
from src import extract_keypoints as extract

# Process all MP4s from a folder and extract labeled keypoints
video_dir = os.path.join("data", "raw")
df = extract.process_directory(video_dir)

# Save result for training
csv_path = os.path.join("data", "pose_data.csv")
extract.save_dataframe(df, csv_path)


Processing left_0.mp4...
Processing left_1.mp4...
Processing left_2.mp4...
Processing neutral_0.mp4...
Processing neutral_1.mp4...
Processing neutral_2.mp4...
Processing right_0.mp4...
Processing right_1.mp4...
Processing right_2.mp4...
Processing up_0.mp4...
Processing up_1.mp4...
Processing up_2.mp4...
Processing wide_0.mp4...
Processing wide_1.mp4...
Processing wide_2.mp4...
✅ Saved keypoints to data\pose_data.csv


# 2. Train Model
create classifier for the keypoints and save the model in models/tf_model

In [3]:
from src import train_pose_model as train
from sklearn.model_selection import train_test_split

model_dir = os.path.join("models", "tf_model")
label_map_path = os.path.join("models", "label_map.json")

# Load data
X, y = train.load_pose_data(csv_path)
y_encoded, label_encoder = train.encode_labels(y)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Build and train model
model = train.build_model(input_dim=X.shape[1], num_classes=len(label_encoder.classes_))
model.summary()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=32)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

# Save model and labels
train.save_model(model, model_dir)
train.save_label_map(label_encoder, label_map_path)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        17,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,605 (100.02 KB)

 Trainable params: 25,605 (100.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3561 - loss: 1.4833 - val_accuracy: 0.8243 - val_loss: 0.8965
Epoch 2/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7124 - loss: 0.8726 - val_accuracy: 0.9287 - val_loss: 0.4623
Epoch 3/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8604 - loss: 0.5119 - val_accuracy: 0.9409 - val_loss: 0.2815
Epoch 4/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9071 - loss: 0.3551 - val_accuracy: 0.9774 - val_loss: 0.1797
Epoch 5/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9442 - loss: 0.2280 - val_accuracy: 0.9861 - val_loss: 0.1171
Epoch 6/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9609 - loss: 0.1765 - val_accuracy: 0.9878 - val_loss: 0.1106
Epoch 7/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9698 - loss: 0.1355 - val_accuracy: 0.9930 - val_loss: 0.0765
Epoch 8/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9796 - loss: 0.1055 - val_accuracy: 0.9896 - val_loss:

INFO:tensorflow:Assets written to: models\tf_model\assets
